通过message方式调用

In [1]:
from http import HTTPStatus
import dashscope
from dashscope import Generation
from dashscope.api_entities.dashscope_response import Role
import os

In [2]:
def conversation_with_messages():
    messages = [{'role': Role.SYSTEM, 'content': 'You are a helpful assistant.'},
                {'role': Role.USER, 'content': '如何做西红柿炖牛腩？'}]
    dashscope.api_key=os.environ['DASHSCOPE_API_KEY']
    response = Generation.call(
        Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        # append result to messages.
        messages.append({'role': response.output.choices[0]['message']['role'],
                         'content': response.output.choices[0]['message']['content']})
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    messages.append({'role': Role.USER, 'content': '不放糖可以吗？'})
    # make second round call
    response = Generation.call(
        Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

In [3]:
conversation_with_messages()

{"status_code": 200, "request_id": "41abd442-b347-95b1-84f5-7f7aaf299a03", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "材料：牛腩1000克，西红柿2个，大葱1根，姜3片，蒜瓣6粒，八角1颗，桂皮1小块，香叶2片，干辣椒2只，生抽2勺，老抽1勺，料酒2勺，冰糖20克，盐适量\n\n做法：\n\n1. 牛腩洗净切块，放沸水中焯水去血沫捞出备用。\n2. 西红柿顶部划十字刀口，放入沸水中烫一下去皮，切滚刀块备用。\n3. 大葱切段、姜切片、蒜瓣剥好。锅内油烧热，下入大葱段、姜片、蒜瓣、八角、桂皮、香叶和干辣椒炒香。\n4. 下入牛腩翻炒均匀，烹入料酒炒匀后，加入生抽和老抽炒至上色。\n5. 加入足量热水（没过牛腩），大火煮开后撇去浮沫，转中小火慢慢炖煮至牛腩软烂，约需1小时以上。\n6. 当牛腩炖至8成熟时，加入西红柿块继续炖煮，炖煮至西红柿软烂且汤汁浓稠即可，最后调入适量的盐调味。\n\n西红柿炖牛腩就做好了！"}}]}, "usage": {"input_tokens": 13, "output_tokens": 286, "total_tokens": 299}}
{"status_code": 200, "request_id": "b4ec10bf-d1e9-9008-82a2-7a07ee51d89f", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "可以的，如果不喜欢甜味，可以不放冰糖或者减少用量。不过需要注意的是，糖能够增加菜肴的鲜美口感和色泽，如果你不放糖，可能会让菜肴的味道变得有些平淡无味。另外，糖还能够中和牛肉的腥味，如果没有糖的话，可能需要在

Prompt 方式调用

In [4]:
# For prerequisites running the following sample, visit https://help.aliyun.com/document_detail/611472.html
from http import HTTPStatus
import dashscope
def call_with_prompt():
    response = dashscope.Generation.call(
        model=dashscope.Generation.Models.qwen_turbo,
        prompt='如何做炒西红柿鸡蛋？'
    )
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if response.status_code == HTTPStatus.OK:
        print(response.output)  # The output text
        print(response.usage)  # The usage information
    else:
        print(response.code)  # The error code.
        print(response.message)  # The error message.



In [5]:
call_with_prompt()

{"text": "材料：鸡蛋2个，西红柿1个。\n\n做法：\n\n1. 鸡蛋打入碗中搅拌均匀备用；\n\n2. 西红柿洗净切块备用；\n\n3. 热锅凉油，油热后放入搅拌好的鸡蛋液，用筷子快速划散成小块，凝固即可盛出备用；\n\n4. 锅内再加少量底油，放入西红柿翻炒出汁，倒入炒好的鸡蛋，加入适量盐、糖调味翻匀即可。", "finish_reason": "stop", "choices": null}
{"input_tokens": 6, "output_tokens": 106, "total_tokens": 112}


流式输出

In [6]:
from http import HTTPStatus
from dashscope import Generation


def call_with_stream():
    messages = [
        {'role': 'user', 'content': '如何做西红柿炖牛腩？'}]
    responses = Generation.call(
        Generation.Models.qwen_turbo,
        messages=messages,
        result_format='message',  # set the result to be "message" format.
        stream=True,
        incremental_output=True  # get streaming output incrementally
    )
    full_content = ''  # with incrementally we need to merge output.
    for response in responses:
        if response.status_code == HTTPStatus.OK:
            full_content += response.output.choices[0]['message']['content']
            print(response)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
    print('Full response:\n' + full_content)


In [7]:
call_with_stream()

{"status_code": 200, "request_id": "8895356b-a620-9103-a5bf-7e8574080867", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "材料"}}]}, "usage": {"input_tokens": 7, "output_tokens": 1, "total_tokens": 8}}
{"status_code": 200, "request_id": "8895356b-a620-9103-a5bf-7e8574080867", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "：牛腩1000克，"}}]}, "usage": {"input_tokens": 7, "output_tokens": 10, "total_tokens": 17}}
{"status_code": 200, "request_id": "8895356b-a620-9103-a5bf-7e8574080867", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "null", "message": {"role": "assistant", "content": "西红柿2个，大葱1根，姜3"}}]}, "usage": {"input_tokens": 7, "output_tokens": 21, "total_tokens": 28}}
{"status_code": 200, "request_id": "8895356b-